## 서브워드 분리(Subword Segmentation)

아무리 많은 단어를 학습시켜도 학습시키지 않은 모르는 단어가 등장할 수 있음

그 단어를 단어 집합에 없는 단어란 의미에서 `OOV(Out-Of-Vocabulary)` 또는 `UNK(Unknown Token)`라고 표현

모르는 단어가 등장하면 문제를 푸는 것이 까다로워 짐

해당 상황을 상황을 OOV 문제라 함

하나의 단어는 더 작은 단위의 의미있는 여러 서브워드들의 조합으로 구성된 경우가 많음

하나의 단어를 여러 서브워드로 분리해서 단어를 인코딩 및 임베딩하겠다는 의도를 가진 전처리 작업을 서브워드 분리(Subword segmenation) 작업 이라 함

OOV나 희귀 단어, 신조어와 같은 문제를 완화시킬 수 있음

실제로 언어의 특성에 따라 영어권 언어나 한국어는 서브워드 분리를 시도했을 때 어느정도 의미있는 단위로 나누는 것이 가능

## 바이트 페어 인코딩(Byte Pair Encoding, BPE)

OOV(Out-Of-Vocabulary) 문제를 완화하는 대표적인 서브워드 분리 알고리즘

BPE(Byte pair encoding) 알고리즘은 1994년에 제안된 데이터 압축 알고리즘, 후에 자연어 처리의 서브워드 분리 알고리즘으로 응용

### BPE의 작동 방식

'aaabdaaabac' 문자열이 주어졌을 때 BPE를 수행하면

BPE은 기본적으로 연속적으로 가장 많이 등장한 글자의 쌍을 찾아서 하나의 글자로 병합하는 방식을 수행

문자열 중 가장 자주 등장하고 있는 바이트의 쌍(byte pair)은 'aa', 이를 하나의 바이트인 'Z'로 치환 하면 다음과 같음

aaabdaaabac

=> ZabdZabac , Z=aa

치환된 문자열 중에서 가장 많이 등장하고 있는 바이트의 쌍은 'ab', 이를 'Y'로 치환 하면

ZabdZabac , Z=aa

=> ZYdZYac , {Y=ab, Z=aa}

반복해서, 치환된 문자열 중에서 가장 많이 등장하고 있는 바이트의 쌍은 'ZY', 이를 'X'로 치환 하면

ZYdZYac , {Y=ab, Z=aa}

=> XdXac , {X=ZY, Y=ab, Z=aa}

더 이상 병합할 바이트의 쌍은 없으므로 BPE는 위의 결과를 최종 결과로 하여 종료

### NLP에서의 BPE(Byte Pair Encoding)

자연어 처리에서의 BPE는 서브워드 분리(subword segmentation) 알고리즘

글자(charcter) 단위에서 점차적으로 단어 집합(vocabulary)을 만들어 내는 Bottom up 방식의 접근을 사용

훈련 데이터에 있는 단어들을 모든 글자(chracters) 또는 유니코드(unicode) 단위로 단어 집합(vocabulary)를 만들고,

가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합

예를 들어 어떤 훈련 데이터로부터 다음과 같이 각 단어들의 빈도수를 카운트하여 딕셔너리를 만들었고 여기에 BPE를 적용하면

low : 5, lower : 2, newest : 6, widest : 3

딕셔너리의 모든 단어들을 글자(chracter) 단위로 분리

딕셔너리는 자신 또한 업데이트되며 앞으로 단어 집합을 업데이트하기 위해 지속적으로 참고되는 참고 자료의 역할

l o w : 5,  l o w e r : 2,  n e w e s t : 6,  w i d e s t : 3

딕셔너리를 참고로 한 초기 단어 집합(vocabulary)은 다음과 같이 글자 단위로 분리된 상태

l, o, w, e, r, n, s, t, i, d

BPE의 특징은 알고리즘의 동작을 몇 회 반복(iteration)할 것인지를 사용자가 정함

10회를 반복한다 가정하면, 각 반복마다 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 수행

1회 - 딕셔너리를 참고로 하였을 때 빈도수가 9로 가장 높은 (e, s)의 쌍을 es로 통합

l o w : 5, l o w e r : 2, n e w es t : 6, w i d es t : 3

l, o, w, e, r, n, s, t, i, d, es

2회 - 빈도수가 9로 가장 높은 (es, t)의 쌍을 est로 통합

l o w : 5, l o w e r : 2, n e w est : 6, w i d est : 3

l, o, w, e, r, n, s, t, i, d, es, est

3회 - 빈도수가 7로 가장 높은 (l, o)의 쌍을 lo로 통합

lo w : 5, lo w e r : 2, n e w est : 6, w i d est : 3

l, o, w, e, r, n, s, t, i, d, es, est, lo

...

최종적으로 얻는 딕셔너리와 단어 집합은 다음과 같음

low : 5, low e r : 2, newest : 6, widest : 3

l, o, w, e, r, n, s, t, i, d, es, est, lo, low, ne, new, newest, wi, wid, widest

테스트 과정에서 'lowest'란 단어가 등장한다면, 기존에는 OOV에 해당되는 단어

BPE 알고리즘을 사용한 위의 단어 집합에서는 더 이상 'lowest'는 OOV가 아님

우선 'lowest'를 전부 글자 단위로 분할하여 'l, o, w, e, s, t'을 얻고 단어 집합을 참고로 하여 'low'와 'est'를 찾아내어

'lowest'를 기계는 'low'와 'est' 두 단어로 인코딩

### 직접 구현

In [8]:
import re, collections
from IPython.display import display, Markdown, Latex

In [9]:
# BPE 반복 횟수
num_merges = 10

In [10]:
dictionary = {'l o w </w>' : 5,
              'l o w e r </w>' : 2,
              'n e w e s t </w>':6,
              'w i d e s t </w>':3}

In [11]:
def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    print('현재 pair들의 빈도수 :', dict(pairs))
    return pairs

In [12]:
def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

In [13]:
bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown(f"### Iteration {i + 1}"))
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print(f"new merge: {best}")
    print(f"dictionary: {dictionary}")

### Iteration 1

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
new merge: ('e', 's')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
new merge: ('es', 't')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('new', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('new', 'est</w>')
dictionary: {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('low', '</w>')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

현재 pair들의 빈도수 : {('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('w', 'i')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


In [14]:
# merge 기록
print(bpe_codes)

{('e', 's'): 0, ('es', 't'): 1, ('est', '</w>'): 2, ('l', 'o'): 3, ('lo', 'w'): 4, ('n', 'e'): 5, ('ne', 'w'): 6, ('new', 'est</w>'): 7, ('low', '</w>'): 8, ('w', 'i'): 9}


In [15]:
def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as a tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs

In [16]:
def encode(orig):
    """Encode word based on list of BPE merge operations, which are applied consecutively"""

    word = tuple(orig) + ('</w>',)
    display(Markdown("__word split into characters:__ <tt>{}</tt>".format(word)))

    pairs = get_pairs(word)    

    if not pairs:
        return orig

    iteration = 0
    while True:
        iteration += 1
        display(Markdown("__Iteration {}:__".format(iteration)))

        print("bigrams in the word: {}".format(pairs))
        bigram = min(pairs, key = lambda pair: bpe_codes.get(pair, float('inf')))
        print("candidate for merging: {}".format(bigram))
        if bigram not in bpe_codes:
            display(Markdown("__Candidate not in BPE merges, algorithm stops.__"))
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word)-1 and word[i+1] == second:
                new_word.append(first+second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        print("word after merging: {}".format(word))
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # 특별 토큰인 </w>는 출력하지 않는다.
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>',''),)

    return word

In [17]:
encode("loki")

__word split into characters:__ <tt>('l', 'o', 'k', 'i', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('i', '</w>'), ('k', 'i'), ('o', 'k'), ('l', 'o')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'k', 'i', '</w>')


__Iteration 2:__

bigrams in the word: {('i', '</w>'), ('lo', 'k'), ('k', 'i')}
candidate for merging: ('i', '</w>')


__Candidate not in BPE merges, algorithm stops.__

('lo', 'k', 'i')

In [18]:
encode("lowest")

__word split into characters:__ <tt>('l', 'o', 'w', 'e', 's', 't', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('t', '</w>'), ('e', 's'), ('l', 'o'), ('w', 'e'), ('o', 'w'), ('s', 't')}
candidate for merging: ('e', 's')
word after merging: ('l', 'o', 'w', 'es', 't', '</w>')


__Iteration 2:__

bigrams in the word: {('t', '</w>'), ('es', 't'), ('l', 'o'), ('o', 'w'), ('w', 'es')}
candidate for merging: ('es', 't')
word after merging: ('l', 'o', 'w', 'est', '</w>')


...

bigrams in the word: {('lo', 'w'), ('w', 'est</w>')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'est</w>')


__Iteration 6:__

bigrams in the word: {('low', 'est</w>')}
candidate for merging: ('low', 'est</w>')


__Candidate not in BPE merges, algorithm stops.__

('low', 'est')

In [19]:
encode("lowing")

__word split into characters:__ <tt>('l', 'o', 'w', 'i', 'n', 'g', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('i', 'n'), ('w', 'i'), ('g', '</w>'), ('l', 'o'), ('o', 'w'), ('n', 'g')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'w', 'i', 'n', 'g', '</w>')


__Iteration 2:__

bigrams in the word: {('i', 'n'), ('w', 'i'), ('lo', 'w'), ('g', '</w>'), ('n', 'g')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'i', 'n', 'g', '</w>')


__Iteration 3:__

bigrams in the word: {('low', 'i'), ('i', 'n'), ('g', '</w>'), ('n', 'g')}
candidate for merging: ('low', 'i')


__Candidate not in BPE merges, algorithm stops.__

('low', 'i', 'n', 'g')

In [20]:
encode("highing")

__word split into characters:__ <tt>('h', 'i', 'g', 'h', 'i', 'n', 'g', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('i', 'g'), ('i', 'n'), ('g', '</w>'), ('g', 'h'), ('n', 'g'), ('h', 'i')}
candidate for merging: ('i', 'g')


__Candidate not in BPE merges, algorithm stops.__

('h', 'i', 'g', 'h', 'i', 'n', 'g')

## BPE 이외의 알고리즘

### WordPiece Tokenizer

WordPiece Tokenizer은 BPE의 변형 알고리즘

BPE가 빈도수에 기반하여 가장 많이 등장한 쌍을 병합하는 것과는 달리, 

병합되었을 때 코퍼스의 우도(Likelihood)를 가장 높이는 쌍을 병합

수행하기 이전의 문장: Jet makers feud over seat width with big orders at stake

WordPiece Tokenizer를 수행한 결과(wordpieces): _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake

모든 단어의 맨 앞에 _를 붙이고, 단어는 서브 워드(subword)로 통계에 기반하여 띄어쓰기로 분리

언더바 _는 문장 복원을 위한 장치, 띄어쓰기의 대용으로 사용

딥 러닝 모델 BERT를 훈련하기 위해서 사용

### Unigram Language Model Tokenizer

유니그램 언어 모델 토크나이저는 각각의 서브워드들에 대해서 손실(loss)을 계산

서브 단어의 손실이라는 것은 해당 서브워드가 단어 집합에서 제거되었을 경우, 코퍼스의 우도(Likelihood)가 감소하는 정도

이렇게 측정된 서브워드들을 손실의 정도로 정렬하여, 최악의 영향을 주는 10~20%의 토큰을 제거

이를 원하는 단어 집합의 크기에 도달할 때까지 반복